In [1]:
###
#   FIG2_3.IPYNB:
#   Creating plots for Figures 2 and 3 in the paper.
###
# By Kirill Sechkar

# IMPORTS
from sr_model import *
import bokeh.io as bkio
bkio.output_notebook()

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

Loading BokehJS ...

In [2]:
# define extents of gene induction considered and magnitude of disturbance
inda_axis = jnp.logspace(jnp.log10(1.8e-4), 0, 51) #jnp.concatenate((jnp.array([0]),jnp.logspace(-4+jnp.log10(1.8), 0, 51)))
inda_axis_np=np.array(inda_axis) # np format for robustness calculations
ind_c_distrubing = 1
ind_f_fbck = 2e-7

# define the range of tested inhibitior gRNA induction extents
indi_axis = jnp.logspace(jnp.log10(1.8e-4), 0, 51)

# initialise model parameter values
par = set_default_pars()

In [3]:
# GET THE DISTURBED AND UNDISTURBED INDUCTION CURVES FOR BASIC CRISPRa
par_basic = par.copy()
par_basic['i_offtarget'] = 1.0
par_basic['alpha_gi'] = 0.0

stacked_mesh_basic = jnp.stack((jnp.zeros_like(inda_axis), inda_axis), axis=1)

# get F values with no competition
x0_no_basic = jnp.concatenate((stacked_mesh_basic, 0 * jnp.ones((len(inda_axis), 1))), axis=1)
p_ss_no_basic_ravel, _, _, F_ss_no_basic_ravel = induction_to_F(x0_no_basic, par_basic)
p_ss_no_basic = np.array(p_ss_no_basic_ravel)
F_ss_no_basic = np.array(F_ss_no_basic_ravel)

# get F values with competition
x0_with_basic = jnp.concatenate((stacked_mesh_basic, ind_c_distrubing * jnp.ones((len(inda_axis), 1))), axis=1)
p_ss_with_basic_ravel, _, _, F_ss_with_basic_ravel = induction_to_F(x0_with_basic, par_basic)
p_ss_with_basic = np.array(p_ss_with_basic_ravel)
F_ss_with_basic = np.array(F_ss_with_basic_ravel)

# calculate performance metrics
dynrange_basic = np.max(p_ss_no_basic) - np.min(p_ss_no_basic) # dynamic range of output gene transcription
fold_change_basic = np.max(p_ss_no_basic) / np.min(p_ss_no_basic)  # fold change upon full induction compared to minimal induction
robustness_basic = rob(p_ss_with_basic, p_ss_no_basic, inda_axis_np)
K_fold_change_basic, K_no_arg_basic, K_with_arg_basic = change_in_K(F_ss_with_basic, F_ss_no_basic, inda_axis_np)

In [4]:
# GET THE DISTURBED AND UNDISTURBED INDUCTION CURVES WITH SR
par_sr=par.copy()

indi_mesh, inda_mesh = jnp.meshgrid(indi_axis, inda_axis)
indi_mesh_ravel = indi_mesh.ravel()
inda_mesh_ravel = inda_mesh.ravel()
stacked_mesh = jnp.stack((indi_mesh_ravel, inda_mesh_ravel), axis=1)

# get F values with no competition
x0_no = jnp.concatenate((stacked_mesh, 0 * jnp.ones((len(inda_mesh_ravel), 1))), axis=1)
p_ss_no_ravel, Fi_ss_no_ravel, Fa_ss_no_ravel, F_ss_no_ravel = induction_to_F(x0_no, par_sr)
p_ss_no = np.array(p_ss_no_ravel.reshape(len(inda_axis), len(indi_axis))).T
F_ss_no = np.array(F_ss_no_ravel.reshape(len(inda_axis), len(indi_axis))).T
Fi_ss_no = np.array(Fi_ss_no_ravel.reshape(len(inda_axis), len(indi_axis))).T
Fa_ss_no = np.array(Fa_ss_no_ravel.reshape(len(inda_axis), len(indi_axis))).T

# get F values with competition
x0_with = jnp.concatenate((stacked_mesh, ind_c_distrubing * jnp.ones((len(inda_mesh_ravel), 1))), axis=1)
p_ss_with_ravel, Fi_ss_with_ravel, Fa_ss_with_ravel, F_ss_with_ravel = induction_to_F(x0_with, par_sr)
p_ss_with = np.array(p_ss_with_ravel.reshape(len(inda_axis), len(indi_axis))).T
F_ss_with = np.array(F_ss_with_ravel.reshape(len(inda_axis), len(indi_axis))).T
Fi_ss_with = np.array(Fi_ss_with_ravel.reshape(len(inda_axis), len(indi_axis))).T
Fa_ss_with = np.array(Fa_ss_with_ravel.reshape(len(inda_axis), len(indi_axis))).T

# calculate performance metrics
dynrange = np.max(p_ss_no,axis=1) - np.min(p_ss_no,axis=1)  # dynamic range of output gene transcription
fold_change = np.max(p_ss_no,axis=1) / np.min(p_ss_no,axis=1)  # fold change upon full induction compared to minimal induction
robustness = rob(p_ss_with, p_ss_no, inda_axis_np) # do not include zero induction!
K_fold_change, K_no_arg, K_with_arg = change_in_K(F_ss_with, F_ss_no, inda_axis_np)

In [5]:
# GET THE DISTURBED AND UNDISTURBED INDUCTION CURVES WITH OFF-TARGET INTERFERENCE
par_off = par.copy()
par_off['i_offtarget'] = 1.0

# get F values with no competition
x0_no_off = jnp.concatenate((stacked_mesh, 0 * jnp.ones((len(inda_mesh_ravel), 1))), axis=1)
p_ss_no_off_ravel, _, _, F_ss_no_off_ravel = induction_to_F(x0_no_off, par_off)
p_ss_no_off = np.array(p_ss_no_off_ravel.reshape(len(inda_axis), len(indi_axis))).T
F_ss_no_off = np.array(F_ss_no_off_ravel.reshape(len(inda_axis), len(indi_axis))).T

# get F values with competition
x0_with_off = jnp.concatenate((stacked_mesh, ind_c_distrubing * jnp.ones((len(inda_mesh_ravel), 1))), axis=1)
p_ss_with_off_ravel, _, _, F_ss_with_off_ravel = induction_to_F(x0_with_off, par_off)
p_ss_with_off = np.array(p_ss_with_off_ravel.reshape(len(inda_axis), len(indi_axis))).T
F_ss_with_off = np.array(F_ss_with_off_ravel.reshape(len(inda_axis), len(indi_axis))).T

# calculate performance metrics
dynrange_off = np.max(p_ss_no_off,axis=1) - np.min(p_ss_no_off,axis=1)  # dynamic range of output gene transcription
fold_change_off = np.max(p_ss_no_off,axis=1) / np.min(p_ss_no_off,axis=1)  # fold change upon full induction compared to minimal induction
robustness_off = rob(p_ss_no_off, p_ss_with_off, inda_axis_np) # do not include zero induction!

In [6]:
# GET THE DISTURBED AND UNDISTURBED INDUCTION CURVES WITH FEEDBACK REGULATION OF dCAS9 SYNTHESIS
par_fbck = par.copy()
par_fbck['fbck_present']=1.0
par_fbck['alpha_gi'] = 0.0 # no co-regulation by the interfering gRNA
par_fbck['alpha_d'] = par['alpha_d']*1e4 # no co-regulation by the interfering gRNA
par_fbck['i_offtarget'] = 1.0 # no co-regulation by the activating gRNA

# no co-regulation by the interfering gRNA
stacked_mesh_fbck = stacked_mesh_basic

# g get F values with no competition
x0_no_fbck = jnp.concatenate((stacked_mesh_fbck, 0 * jnp.ones((len(inda_axis), 1))), axis=1)
p_ss_no_fbck_ravel, _, _, F_ss_no_fbck_ravel = induction_to_F(x0_no_fbck, par_fbck)
p_ss_no_fbck = np.array(p_ss_no_fbck_ravel)
F_ss_no_fbck = np.array(F_ss_no_fbck_ravel)

# get F values with competition
x0_with_fbck = jnp.concatenate((stacked_mesh_fbck, ind_c_distrubing * jnp.ones((len(inda_axis), 1))), axis=1)
p_ss_with_fbck_ravel, _, _, F_ss_with_fbck_ravel = induction_to_F(x0_with_fbck, par_fbck)
p_ss_with_fbck = np.array(p_ss_with_fbck_ravel)
F_ss_with_fbck = np.array(F_ss_with_fbck_ravel)

# calculate performance metrics
dynrange_fbck = np.max(p_ss_no_fbck) - np.min(p_ss_no_fbck)  # dynamic range of output gene transcription
fold_change_fbck = np.max(p_ss_no_fbck) / np.min(p_ss_no_fbck)  # fold change upon full induction compared to minimal induction
robustness_fbck = rob(p_ss_with_fbck, p_ss_no_fbck, inda_axis_np)

In [7]:
# FIG 2C,D: INDUCTION CURVES WITHOUT AND WITH DISTURBANCE

bkplot.output_backend = "svg"
plot_x_range= (5e-6, 1)

# ind_i value for example dose-response curve
sr_example_indi = 1e-2

# plot single-activator dose-response curve
single_curve_fig = bkplot.figure(frame_width=256, frame_height=144,
                                 x_axis_label='ua, induction of activating gRNA gene',
                                 y_axis_label="p, output prot. conc. [nM]",
                                 x_axis_type='log', tools='pan,box_zoom,hover,reset,save',
                                 x_range=(1.8e-4,1),
                                 y_range=(0,2.1e4)
                                 )
single_curve_fig.output_backend = "svg"
# curves
single_curve_fig.line(np.array(inda_axis),
                      p_ss_no_basic,
                      line_width=2, line_color='#de3163ff', legend_label="p(ua,uc=0)")
single_curve_fig.line(inda_axis_np, p_ss_with_basic,
                      line_width=2, line_color='#de3163ff', line_dash='dashed',
                      legend_label='p(ua,uc=' + str(ind_c_distrubing)+')')
# legend
single_curve_fig.legend.location = "top_left"
single_curve_fig.legend.border_line_color = "gray"
single_curve_fig.legend.border_line_alpha = 0.3
single_curve_fig.legend.background_fill_color = "white"
single_curve_fig.legend.background_fill_alpha = 1
single_curve_fig.legend.margin=2
single_curve_fig.legend.padding=3
single_curve_fig.legend.spacing=0
#fonts
single_curve_fig.title.text_font_size = "8pt"
single_curve_fig.xaxis.axis_label_text_font_size = "8pt"
single_curve_fig.yaxis.axis_label_text_font_size = "8pt"
single_curve_fig.xaxis.major_label_text_font_size = "8pt"
single_curve_fig.yaxis.major_label_text_font_size = "8pt"
single_curve_fig.legend.label_text_font_size = "8pt"

# plot the dose-response curve with SR
indi_example_index = np.where(np.array(indi_axis) >= sr_example_indi)[0][0]
sr_curve_fig = bkplot.figure(frame_width=256, frame_height=144,
                                 x_axis_label='ua, induction of activating gRNA gene',
                                 y_axis_label="p, output prot. conc. [nM]",
                                 x_axis_type='log', tools='pan,box_zoom,hover,reset,save',
                                 x_range=(1.8e-4,1),
                                y_range=(0,2.1e4)
                                 )
sr_curve_fig.output_backend = "svg"
#curves
# sr_curve_fig.line(np.array(inda_axis[1:]), p_ss_no[indi_example_index, 1:],
#                   line_width=2, line_color='#ff6700ff', legend_label='u_c =0')
sr_curve_fig.line(inda_axis_np, p_ss_no[indi_example_index,:],
                  line_width=2, line_color='#ff6700ff', legend_label='p(ua,uc=0)')
sr_curve_fig.line(inda_axis_np, p_ss_with[indi_example_index,:],
                  line_width=2, line_color='#ff6700ff', line_dash='dashed',
                  legend_label='p(ua,uc=' + str(ind_c_distrubing)+')')
# legend
sr_curve_fig.legend.location = "top_left"
sr_curve_fig.legend.border_line_color = "gray"
sr_curve_fig.legend.border_line_alpha = 0.3
sr_curve_fig.legend.background_fill_color = "white"
sr_curve_fig.legend.background_fill_alpha = 1
sr_curve_fig.legend.margin=2
sr_curve_fig.legend.padding=3
sr_curve_fig.legend.spacing=0
#fonts
sr_curve_fig.title.text_font_size = "8pt"
sr_curve_fig.xaxis.axis_label_text_font_size = "8pt"
sr_curve_fig.yaxis.axis_label_text_font_size = "8pt"
sr_curve_fig.xaxis.major_label_text_font_size = "8pt"
sr_curve_fig.yaxis.major_label_text_font_size = "8pt"
sr_curve_fig.legend.label_text_font_size = "8pt"

# show plots
bkplot.show(bklayouts.grid([[single_curve_fig, sr_curve_fig]]))

# print half-saturation point changes
print('Single activator: half-saturation point change: '+str(max(K_fold_change_basic,1/K_fold_change_basic))+'-fold')
print('SR: half-saturation point change: '+str(max(K_fold_change[indi_example_index],1/K_fold_change[indi_example_index]))+'-fold')

Single activator: half-saturation point change: 15.787479453957054-fold
SR: half-saturation point change: 2.8142799679711636-fold


In [8]:
# FIG 3C,D: ROBUSTNESS TO DISTURBANCE AND OUTPUT FOLD-CHANGE

# plot the maxium induction of output protein
maxF_fig = bkplot.figure(width=348, height=261, title="Max. induction of output prot. exp.",
                         x_axis_label='u_i', y_axis_label="F, output gene transc. reg.",
                         x_axis_type='log', tools='pan,box_zoom,hover,reset,save')
maxF_fig.line(np.array(indi_axis), F_ss_no_basic[-1],
              line_width=2, line_color='#de3163ff', legend_label='Basic')
maxF_fig.line(np.array(indi_axis), F_ss_no[:, -1],
              line_width=2, line_color='#ff6700ff', legend_label='SR')
maxF_fig.line(np.array(indi_axis), F_ss_no_off[:, -1],
              line_width=2, line_color='#48d1ccff', legend_label='Off-target')
maxF_fig.line(np.array(indi_axis), F_ss_no_fbck[-1],
              line_width=2, line_color='#32cd32ff', legend_label='Feedback')
maxF_fig.legend.location = "bottom_left"
# show which value of ind_i corresponds to the example curve
maxF_fig.add_layout(bkmodels.Span(location=sr_example_indi,
                                  dimension='height', line_color='black', line_dash='dotted', line_width=2))
maxF_fig.output_backend = "svg"

# plot the output dynamic range
dynrange_fig = bkplot.figure(frame_width=256, frame_height=144,
                               x_axis_label='ui, induction of interfering gRNA gene', y_axis_label="Δp, output range [nM]",
                               x_range=(1.8e-4,1),
                             # y_range=(0, 20000),
                               x_axis_type='log',
                               tools='pan,box_zoom,hover,reset,save')
dynrange_fig.output_backend = "svg"
#curves
dynrange_fig.line(np.array(indi_axis), dynrange_basic,
                  line_width=2, line_color='#de3163ff', legend_label='Basic')
dynrange_fig.line(np.array(indi_axis), dynrange,
                  line_width=2, line_color='#ff6700ff', legend_label='SR')
dynrange_fig.line(np.array(indi_axis), dynrange_off,
                  line_width=2, line_color='#48d1ccff', legend_label='Off-target')
dynrange_fig.line(np.array(indi_axis), dynrange_fbck,
                  line_width=2, line_color='#32cd32ff', legend_label='Feedback')
dynrange_fig.legend.location = "bottom_left"
# show which value of ind_i corresponds to the example curve
dynrange_fig.add_layout(bkmodels.Span(location=sr_example_indi,
                                      dimension='height', line_color='black', line_dash='dotted', line_width=2))
# legend
dynrange_fig.legend.location = "bottom_left"
dynrange_fig.legend.border_line_color = "gray"
dynrange_fig.legend.border_line_alpha = 0.3
dynrange_fig.legend.background_fill_color = "white"
dynrange_fig.legend.background_fill_alpha = 1
dynrange_fig.legend.margin=2
dynrange_fig.legend.padding=3
dynrange_fig.legend.spacing=0
# ticks
dynrange_fig.xaxis.ticker = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
# dynrange_fig.yaxis.ticker = np.logspace(-9,0,4)
# fonts
dynrange_fig.title.text_font_size = "8pt"
dynrange_fig.xaxis.axis_label_text_font_size = "8pt"
dynrange_fig.yaxis.axis_label_text_font_size = "8pt"
dynrange_fig.xaxis.major_label_text_font_size = "8pt"
dynrange_fig.yaxis.major_label_text_font_size = "8pt"
dynrange_fig.legend.label_text_font_size = "8pt"

# plot the fold change
fold_change_fig = bkplot.figure(frame_width=256, frame_height=144,
                               x_axis_label='ui, induction of interfering gRNA gene', y_axis_label="Output prot. conc. fold-change",
                               x_range=(1.8e-4,1), y_range=(10,1e5),
                               x_axis_type='log', y_axis_type='log',
                               tools='pan,box_zoom,hover,reset,save')
fold_change_fig.output_backend = "svg"
# show which value of ind_i corresponds to the example curve
fold_change_fig.add_layout(bkmodels.Span(location=sr_example_indi,
                                         dimension='height', line_color='black', line_dash='dotted', line_width=2))
# curves
fold_change_fig.line(np.array(indi_axis), fold_change_basic,
                     line_width=2, line_color='#de3163ff', legend_label='Basic')
fold_change_fig.line(np.array(indi_axis), fold_change,
                     line_width=2, line_color='#ff6700ff', legend_label='SR')
fold_change_fig.line(np.array(indi_axis), fold_change_off,
                     line_width=2, line_color='#48d1ccff', legend_label='Off-target')
fold_change_fig.line(np.array(indi_axis), fold_change_fbck,
                     line_width=2, line_color='#32cd32ff', legend_label='dCas9 fbck')
fold_change_fig.legend.location = "top_left"
# legend
fold_change_fig.legend.location = "top_left"
fold_change_fig.legend.border_line_color = "gray"
fold_change_fig.legend.border_line_alpha = 0.3
fold_change_fig.legend.background_fill_color = "white"
fold_change_fig.legend.background_fill_alpha = 1
fold_change_fig.legend.margin=2
fold_change_fig.legend.padding=2
fold_change_fig.legend.spacing=0
# ticks
fold_change_fig.xaxis.ticker = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
fold_change_fig.yaxis.ticker = np.logspace(1,5,5)
# fonts
fold_change_fig.title.text_font_size = "8pt"
fold_change_fig.xaxis.axis_label_text_font_size = "8pt"
fold_change_fig.yaxis.axis_label_text_font_size = "8pt"
fold_change_fig.xaxis.major_label_text_font_size = "8pt"
fold_change_fig.yaxis.major_label_text_font_size = "8pt"
fold_change_fig.legend.label_text_font_size = "8pt"
fold_change_fig.legend.label_text_font_size = "8pt"

# plot the robustness
robustness_fig = bkplot.figure(frame_width=256, frame_height=144,
                               x_axis_label='ui, induction of interfering gRNA gene', y_axis_label="ρ, robustness to comp.[nM-2]",
                               x_range=(1.8e-4,1),
                               y_range=(1e-9,1),
                               x_axis_type='log', y_axis_type='log',
                               tools='pan,box_zoom,hover,reset,save')
robustness_fig.output_backend = "svg"
# show which value of ind_i corresponds to the example curve
robustness_fig.add_layout(bkmodels.Span(location=sr_example_indi,
                                        dimension='height', line_color='black', line_dash='dotted', line_width=2))
# curves
robustness_fig.line(np.array(indi_axis), robustness_basic,
                    line_width=2, line_color='#de3163ff', legend_label='Basic')
robustness_fig.line(np.array(indi_axis), robustness,
                    line_width=2, line_color='#ff6700ff', legend_label='SR')
robustness_fig.line(np.array(indi_axis), robustness_off,
                    line_width=2, line_color='#48d1ccff', legend_label='Off-target')
robustness_fig.line(np.array(indi_axis), robustness_fbck,
                    line_width=2, line_color='#32cd32ff', legend_label='dCas9 fbck')
# legend
robustness_fig.legend.location = "top_left"
robustness_fig.legend.border_line_color = "gray"
robustness_fig.legend.border_line_alpha = 0.3
robustness_fig.legend.background_fill_color = "white"
robustness_fig.legend.background_fill_alpha = 1
robustness_fig.legend.margin=2
robustness_fig.legend.padding=2
robustness_fig.legend.spacing=0
# ticks
robustness_fig.xaxis.ticker = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
robustness_fig.yaxis.ticker = np.logspace(-9,-1,5)
# fonts
robustness_fig.title.text_font_size = "8pt"
robustness_fig.xaxis.axis_label_text_font_size = "8pt"
robustness_fig.yaxis.axis_label_text_font_size = "8pt"
robustness_fig.xaxis.major_label_text_font_size = "8pt"
robustness_fig.yaxis.major_label_text_font_size = "8pt"
robustness_fig.legend.label_text_font_size = "8pt"

# show plots
bkplot.show(bklayouts.grid([[robustness_fig, fold_change_fig]]))